<a href="https://colab.research.google.com/github/martians-sheep/pl_task_recomended_csd/blob/main/elastic_more_like_this.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elasticsearchでmore like thisによる類似検索を実行するサンプル
## 前提条件
このサンプルコードではElasticsearchはv8.13.0を使用します。
またElasticsearchを外部に作成し、そこへアクセスする形となります。

In [ ]:
# Elasticsearch ライブラリのインストール
!pip install elasticsearch
from elasticsearch import Elasticsearch

In [10]:
# Elasticsearchのエンドポイントとポート番号を指定
es_endpoint = "{Elasticsearchのエンドポイント}"
username = "{username} ex. elasticなど"
password = "{password}"

In [ ]:
# Elasticsearchクライアントの作成
try:

    # Elasticsearchクライアントの作成
    es = Elasticsearch(
        hosts=[es_endpoint],
        http_auth=(username,password)
    )

    # Elasticsearchへの接続を確認
    if es.ping():
        print("Elastic Searchへの接続が確認されました。")
    else:
        print("Elastic Searchへの接続に失敗しました。")

except Exception as e:
    print(f"Elastic Searchへの接続中にエラーが発生しました: {e}")

In [ ]:
# Elasticsearchの情報表示
print(es.info())

# ドキュメントの取得とインデックス登録

In [ ]:
index_name = "more_like_this_doc"

# インデックスの設定
index_settings = {
    "settings": {
        "analysis": {
            "tokenizer": {
                "kuromoji_tokenizer": {
                    "type": "kuromoji_tokenizer",
                    "mode": "search",
                    "discard_compound_token": True
                }
            },
            "filter": {
                "kuromoji_filter": {
                    "type": "kuromoji_part_of_speech",
                    "stoptags": ["助詞-格助詞-一般", "助詞-終助詞"]
                }
            },
            "analyzer": {
                "kuromoji_analyzer": {
                    "type": "custom",
                    "tokenizer": "kuromoji_tokenizer",
                    "filter": [
                        "kuromoji_baseform",
                        "kuromoji_filter",
                        "cjk_width",
                        "ja_stop",
                        "kuromoji_stemmer",
                        "lowercase"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "title": {
                "type": "text",
                "analyzer": "kuromoji_analyzer"
            },
            "content": {
                "type": "text",
                "analyzer": "kuromoji_analyzer"
            }
        }
    }
}

# インデックスの作成
es.indices.create(index=index_name, body=index_settings)

In [ ]:
# ドキュメントのパスをGoogle Driveから取得
from google.colab import drive
import os

# Google Driveをマウントする
drive.mount('/content/drive')

# 起点となるフォルダのパスを指定する
base_folder = '/content/drive/MyDrive/PL自動割り当て用プロトデータ'

def get_file_paths(folder_path):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):  # テキストファイルのみ処理する
                file_path = os.path.join(root, file)
                file_paths.append(file_path)
    return file_paths

# 起点となるフォルダ内のテキストファイルのパスを取得する
file_paths = get_file_paths(base_folder)

# ファイルパスを表示する
print(len(file_paths))

In [33]:
# forループでインデックス登録
def add_index_docs(file_paths):
   for file_path in file_paths:
        print(file_path)
        # ファイルの内容を読み取る
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()

        # ファイル名をタイトルとして取得
        title = os.path.basename(file_path)

        # ドキュメントを作成
        document = {
            "title": title,
            "content": content
        }

        # ドキュメントのインデックス化
        es.index(index=index_name, body=document)


In [ ]:
# インデックス登録
add_index_docs(file_paths)

print("end")

# 類似検索の実行

In [41]:
# 類似文書を検索するためのクエリ

# テキストを読み込む
txt_file_path = "./法律AIの活用に関する研究報告書.txt"

with open(txt_file_path, "r", encoding="utf-8") as file:
    in_text = file.read()

query = {
    "more_like_this": {
        "fields": ["content"],  # 類似性を判断するフィールド
        "like": in_text,  # 類似文書を検索するための入力テキスト
        "min_term_freq": 1,  # 最小ターム頻度
        "max_query_terms": 12  # クエリで使用する最大ターム数
    }
}

# More Like Thisクエリの実行
response = es.search(index=index_name, body={"query": query})

# 検索結果の表示
print("検索結果:")
for hit in response["hits"]["hits"]:
    print("タイトル:", hit["_source"]["title"])
    print("スコア:", hit["_score"])
    print("本文:", hit["_source"]["content"])
    print("---")

検索結果:
タイトル: response_2024-03-28_09-07-33.txt
スコア: 45.484066
本文: 
タイトル: 法律AI活用に関する研究報告書

1. 研究の背景と目的
法律分野におけるAI技術の進化により、法律業界における業務効率化やリスク管理の向上が期待されています。本研究の目的は、国内外の事例を調査し、法律AIの活用可能性と課題を明らかにし、将来展望を報告することです。

2. 研究方法
本研究では、国内外の法律事務所や法律関連企業がAI技術を活用した事例を調査し、それぞれの取り組みや成果を分析しました。さらに、AI技術の専門家や法律業界の専門家へのインタビュー調査も行い、法律AIの現状と将来性についての情報を収集しました。

3. 研究結果
調査の結果、法律分野におけるAI技術の活用は、契約書の自動作成や法的リサーチの効率化、法的リスクの予測など、さまざまな分野で効果が確認されています。一方で、精度やプライバシー保護などの課題も指摘されています。

4. 考察と今後の展望
本研究から、法律AIの活用は法律業界において効果的であり、今後もさらなる進化が期待されるということが明らかになりました。しかし、精度や倫理的な問題などの課題もあるため、これらを解決するための取り組みが必要とされています。今後も法律AIの研究と開発を進め、法的業務の効率化や品質向上に貢献していくことが重要であると考えられます。
---
タイトル: response_2024-03-28_08-57-18.txt
スコア: 44.209297
本文: 法律AI活用に関する研究報告書

1. 研究の背景と目的
近年、AI技術の急速な発展により、法律分野においてもAIの活用が進んでいる。本研究では、国内外の法律AI活用の事例を調査し、その可能性と課題について明らかにすることを目的とする。

2. 研究方法
研究では、国内外の法律AI活用の事例を分析し、その効果や課題について整理するために、文献調査とインタビュー調査を実施した。また、法律AI技術の最新動向についても研究し、将来展望を考察した。

3. 研究結果
研究の結果、法律AI技術は契約書作成や労働法違反の検出など、特定の業務領域において高い効率化と精度向上が期待される一方で、倫理的な問題やデータの信頼性、プライバ

# bacthでドキュメント登録(上手くいかなかった)

In [28]:
from elasticsearch import helpers

def batch_generator(file_paths, batch_size):
    bulk_data = []
    for file_path in file_paths:
        # ファイルの内容を読み取る
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()

        # ファイル名をタイトルとして取得
        title = os.path.basename(file_path)

        # ドキュメントを作成
        document = {
            "title": title,
            "content": content
        }

        # アクション辞書とドキュメントデータの辞書のペアを追加
        bulk_data.append({"index": {"_index": index_name}})
        bulk_data.append(document)

        if len(bulk_data) == batch_size * 2:
            print("Yielding bulk data:", bulk_data)  # デバッグ用の出力
            yield bulk_data
            bulk_data = []

    if bulk_data:
        print("Yielding remaining bulk data:", bulk_data)  # デバッグ用の出力
        yield bulk_data

In [ ]:
# 一度に登録する数
batch_size = 20

# バッチ処理でバルクインデックス化を実行
success_count = 0
for batch in batch_generator(file_paths, batch_size):
    success, _ = helpers.bulk(es, batch)
    success_count += success
    print(f"Indexed {success} documents.")

print(f"Total indexed documents: {success_count}")